# Código em Python - Projeto de Banco de Dados

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Teste12345",
    database="escola"
)
mycursor = mydb.cursor()

def imprime_result(myresult,titulo,tit_registro):
    codigo = 0
    print("\n","-"*25,titulo,"-"*25)
    for registro in myresult:
        if registro[0] != codigo:
            if codigo != 0:
                print("-"*80)
            codigo = registro[0]
            print(f"{tit_registro}: {codigo} - {registro[1]}")
        print(f"\t{registro[2]:>2} - {registro[3]}")
    input("\n===> Tecle ENTER para continuar")
    return

def alunos_curso():
    sql = "SELECT matricula.cur_codigo ,cur_nome, matricula.alu_cpf, alu_nome, alu_endereco,\
        alu_pais, alu_ddd, alu_fone, alu_ramal, DATE_FORMAT(alu_dtnasc, '%d/%m/%Y') \
        FROM matricula \
        INNER JOIN curso ON (matricula.cur_codigo = curso.cur_codigo) \
        INNER JOIN aluno ON (matricula.alu_cpf = aluno.alu_cpf) \
        ORDER BY cur_nome, alu_nome"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    print("-"*25," Alunos Matriculados nos Cursos ","-"*25)
    codigo = 0
    for registro in myresult:
        if registro[0] != codigo:
            if codigo != 0:
                print("-"*60)
            codigo = registro[0]
            print(f"Curso: {codigo}-{registro[1]}")
        print(f"\t{registro[2]} - {registro[3]:<40} \tNascimento: {registro[9]} "
              f"\n\t\t{registro[4]:<50} \tFone:{registro[5]}({registro[6]}){registro[7]} R:{registro[8]} ")
    input("\n===> Tecle ENTER para continuar")
    return

def disciplinas_curso():
    sql = "SELECT composicao.cur_codigo AS 'Cod', cur_nome AS 'Curso', composicao.dis_codigo AS 'Cod', \
            dis_nome AS 'Disciplina', dis_qtde_creditos AS 'Cred' \
            FROM composicao \
            INNER JOIN curso ON (composicao.cur_codigo = curso.cur_codigo) \
            INNER JOIN disciplina ON (composicao.dis_codigo = disciplina.dis_codigo) \
            ORDER BY cur_nome"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    imprime_result(myresult," Disciplinas dos Cursos ","Curso")
    return

def disciplinas_aluno():
    sql = "SELECT disc_aluno.alu_cpf AS 'CPF', alu_nome AS 'Aluno', disc_aluno.dis_codigo AS 'Cod', \
        dis_nome AS 'Disciplina', dis_qtde_creditos AS 'Cred' \
        FROM disc_aluno \
        INNER JOIN aluno ON (disc_aluno.alu_cpf = aluno.alu_cpf) \
        INNER JOIN disciplina ON (disc_aluno.dis_codigo = disciplina.dis_codigo)\
        ORDER BY alu_nome"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    imprime_result(myresult," Disciplinas do aluno ","Aluno")
    return

def disciplinas_professor():
    sql = "SELECT professor.pro_matricula, pro_nome, dis_codigo, dis_nome, dis_qtde_creditos, \
        pro_endereco, pro_pais, pro_ddd, pro_fone, pro_ramal, DATE_FORMAT(pro_dtnasc, '%d/%m/%Y'), \
        professor.dep_codigo, dep_nome, DATE_FORMAT(pro_dt_contrato, '%d/%m/%Y') \
        FROM disciplina \
        INNER JOIN professor ON (disciplina.pro_matricula = professor.pro_matricula) \
        INNER JOIN departamento ON (professor.dep_codigo = departamento.dep_codigo) \
        ORDER BY pro_nome"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    print("-"*29," Disciplinas do Professor ","-"*29)
    codigo = 0
    for registro in myresult:
        if registro[0] != codigo:
            if codigo != 0:
                print("-"*86)
            codigo = registro[0]
            print(f"Professor: {codigo:>3} - {registro[1]:<40} Depto: {registro[11]} {registro[12]}({registro[13]})")
            print(f"\t\t{registro[5]:<40} \t\t  Fone:{registro[6]}({registro[7]}){registro[8]} R:{registro[9]}\n"
                  f"\t\tNasc: {registro[10]}\n")
        print(f"\tDisciplina: {registro[2]} - {registro[3]:<40} \tCréditos: {registro[4]} ")
    input("\n===> Tecle ENTER para continuar")
    return


def pre_disciplinas():
    sql = "SELECT pre_req.dis_codigo, d1.dis_nome, pre_req.dis_codigo_dep, d2.dis_nome \
        FROM pre_req \
        INNER JOIN disciplina d1 ON (pre_req.dis_codigo = d1.dis_codigo) \
        INNER JOIN disciplina d2 ON (pre_req.dis_codigo_dep = d2.dis_codigo) \
        ORDER BY d1.dis_nome"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    imprime_result(myresult," Pré-requisitos das Disciplinas ","Disciplina")
    return

def media_idade():
    sql = "SELECT matricula.cur_codigo AS 'Cod', cur_nome as 'Curso', \
        format(AVG(FLOOR(DATEDIFF(NOW(), alu_dtnasc)/365)),'N') AS 'Idade Média' \
        from matricula \
        INNER JOIN curso ON (matricula.cur_codigo = curso.cur_codigo) \
        INNER JOIN aluno ON (matricula.alu_cpf = aluno.alu_cpf) \
        GROUP BY matricula.cur_codigo"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    print("-"*20," Media de Idade dos Alunos ","-"*20)
    for registro in myresult:
        print(f"Curso: {registro[0]:>2} - {registro[1]:<30}  Idade Média:{registro[2]}")
    input("\n===> Tecle ENTER para continuar")
    return

def depto_cursos():
    sql = "SELECT departamento.dep_codigo, dep_nome, COALESCE(cur_codigo, '-'), COALESCE(cur_nome,'* Sem Oferta de Curso *') \
        FROM curso \
        RIGHT JOIN departamento ON (curso.dep_codigo = departamento.dep_codigo) \
        ORDER BY dep_nome"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    imprime_result(myresult," Cursos oferecidos por Departamento ","Departamento")
    return

titulo = "Menu Principal"
while True:
    print("="*80)
    print(f"{titulo:^80}")
    print("="*80)
    print("1 - Relatório de Alunos matriculados nos cursos")
    print("2 - Relatório de Disciplinas de cada curso")
    print("3 - Relatório de Disciplinas de cada aluno")
    print("4 - Relatório de Disciplinas de cada professor")
    print("5 - Relatório de Pré-requisitos das disciplinas")
    print("6 - Relatório de Média de Idade dos alunos por curso")
    print("7 - Relatório de Cursos oferecidos por departamento")
    print("0 - Encerrar")

    opcao = input("\nInforme a sua opção: ")

    if opcao == "0":
        break
    elif opcao == "1":
        alunos_curso()
    elif opcao == "2":
        disciplinas_curso()
    elif opcao == "3":
        disciplinas_aluno()
    elif opcao == "4":
        disciplinas_professor()
    elif opcao == "5":
        pre_disciplinas()
    elif opcao == "6":
        media_idade()
    elif opcao == "7":
        depto_cursos()
    else:
        print("===> Opção Inválida...")

## Respostas:


### 1) Produza um relatório que contenha os dados dos alunos matriculados em todos os cursos oferecidos pela escola

### 2)Produza um relatório com os dados de todos os cursos, com suas respectivas disciplinas, oferecidos pela escola

### 3) Produza um relatório que contenha o nome dos alunos e as disciplinas em que estão matriculados

### 4) Produza um relatório com os dados dos professores e as disciplinas que ministram

### 5) Produza um relatório com os nomes das disciplinas e seus pré-requisitos

### 6) Produza um relatório com a média de idade dos alunos matriculados em cada curso

### 7) Produza um relatório com os cursos oferecidos por cada departamento